In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist, squareform

In [2]:
dfmeta = pd.read_csv("product.csv")
dfmeta.rename(columns={'id': 'productid', 'name':'title'}, inplace=True)
dfmeta.head()

,productid,title,sku,description,short_description,thumbnail_url,brand_id,price,is_active,is_published,updated_at,created_at
0,124742926,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,4409272649267,<h3>Tiêu đề</h3><p>Mạnh mẽ vươn tới thành công...,Tiêu đềMạnh mẽ vươn tới thành công. Mà nhẹ bỗn...,https://salt.tikicdn.com/cache/280x280/media/c...,17827,22900000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
1,74556865,Máy Tính Xách Tay Laptop Huawei Matebook D15 (...,7176829303455,<div><strong>MateBook D15 </strong></div><div>...,MateBook D15 MateBook D15 i3 / R7 MateBook ...,https://salt.tikicdn.com/cache/280x280/ts/prod...,19788,11480000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
2,144923785,Apple MacBook Pro 2021 14-inch (Apple M1 PRO -...,6074391823390,<h3>Tiêu đề</h3><p>MacBook Pro. Mang Pro đi họ...,Tiêu đềMacBook Pro. Mang Pro đi học.MacBook Pr...,https://salt.tikicdn.com/cache/280x280/ts/prod...,17827,48200000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
3,125013938,Apple MacBook Pro 2020 13-inch (Apple M1 - 8GB...,5588261800174,<style type=text/css><!--td {border: 1px solid...,Tiêu đềSức mạnh để tiến xa hơn trong học tập.M...,https://salt.tikicdn.com/cache/280x280/ts/prod...,17827,30100000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
4,125897193,Laptop Lenovo IP 1 11IGL05 81VT006FVN (Pentium...,3752128620310,"<h5>Thiết kế gọn gàng, chắc chắn </h5><p><stro...","Thiết kế gọn gàng, chắc chắn Laptop Lenovo IP ...",https://salt.tikicdn.com/cache/280x280/ts/prod...,18604,8890000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01


In [3]:
dfreviews = pd.read_csv("product_rating.csv", dtype={"star_number": "int8", "user_id": "int32", "product_id": "int32"})
dfreviews.rename(columns={'product_id': 'productid', 'star_number': 'rating', 'user_id': 'userid'}, inplace=True)
dfreviews.head()

,id,productid,userid,comment,rating,updated_at,created_at
0,16757337,124742926,1110677,"Hàng mới (Sản xuất 2022) nguyên seal, do hàng ...",5,2022-06-18 19:04:49,2022-06-18 19:04:49
1,14766918,124742926,5249922,"Tình hình là sau 6 ngày sử dụng, máy hoàn hão ...",1,2022-01-31 10:29:58,2022-01-31 10:29:58
2,12133904,124742926,1464242,Mình đặt máy Mac Air M1 16Gb Ram - 256 SSD giá...,5,2021-09-08 20:58:20,2021-09-08 20:58:20
3,11648039,124742926,299031,"Hàng chình hãng, nguyên seal, máy mới và đẹp. ...",4,2021-08-06 23:23:46,2021-08-06 23:23:46
4,7608910,124742926,137665,"- Đóng gói chắc chắn, giao hàng nhanh - Sản ph...",5,2021-02-07 18:55:32,2021-02-07 18:55:32


## Đánh trọng số các đánh giá tốt: 5* x hệ số 2; 4* x hệ số 1.5



In [4]:
dfreviews.loc[dfreviews['rating'] == 5, 'rating'] *= 2
dfreviews.loc[dfreviews['rating'] == 4, 'rating'] *= 1.5
dfreviews1 = dfreviews
dfreviews1.head()

,id,productid,userid,comment,rating,updated_at,created_at
0,16757337,124742926,1110677,"Hàng mới (Sản xuất 2022) nguyên seal, do hàng ...",10,2022-06-18 19:04:49,2022-06-18 19:04:49
1,14766918,124742926,5249922,"Tình hình là sau 6 ngày sử dụng, máy hoàn hão ...",1,2022-01-31 10:29:58,2022-01-31 10:29:58
2,12133904,124742926,1464242,Mình đặt máy Mac Air M1 16Gb Ram - 256 SSD giá...,10,2021-09-08 20:58:20,2021-09-08 20:58:20
3,11648039,124742926,299031,"Hàng chình hãng, nguyên seal, máy mới và đẹp. ...",6,2021-08-06 23:23:46,2021-08-06 23:23:46
4,7608910,124742926,137665,"- Đóng gói chắc chắn, giao hàng nhanh - Sản ph...",10,2021-02-07 18:55:32,2021-02-07 18:55:32


In [5]:
product_ratings=pd.merge(dfreviews,dfmeta,on='productid',how='inner')
product_ratings.head()

,id,productid,userid,comment,rating,updated_at_x,created_at_x,title,sku,description,short_description,thumbnail_url,brand_id,price,is_active,is_published,updated_at_y,created_at_y
0,16757337,124742926,1110677,"Hàng mới (Sản xuất 2022) nguyên seal, do hàng ...",10,2022-06-18 19:04:49,2022-06-18 19:04:49,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,4409272649267,<h3>Tiêu đề</h3><p>Mạnh mẽ vươn tới thành công...,Tiêu đềMạnh mẽ vươn tới thành công. Mà nhẹ bỗn...,https://salt.tikicdn.com/cache/280x280/media/c...,17827,22900000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
1,14766918,124742926,5249922,"Tình hình là sau 6 ngày sử dụng, máy hoàn hão ...",1,2022-01-31 10:29:58,2022-01-31 10:29:58,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,4409272649267,<h3>Tiêu đề</h3><p>Mạnh mẽ vươn tới thành công...,Tiêu đềMạnh mẽ vươn tới thành công. Mà nhẹ bỗn...,https://salt.tikicdn.com/cache/280x280/media/c...,17827,22900000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
2,12133904,124742926,1464242,Mình đặt máy Mac Air M1 16Gb Ram - 256 SSD giá...,10,2021-09-08 20:58:20,2021-09-08 20:58:20,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,4409272649267,<h3>Tiêu đề</h3><p>Mạnh mẽ vươn tới thành công...,Tiêu đềMạnh mẽ vươn tới thành công. Mà nhẹ bỗn...,https://salt.tikicdn.com/cache/280x280/media/c...,17827,22900000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
3,11648039,124742926,299031,"Hàng chình hãng, nguyên seal, máy mới và đẹp. ...",6,2021-08-06 23:23:46,2021-08-06 23:23:46,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,4409272649267,<h3>Tiêu đề</h3><p>Mạnh mẽ vươn tới thành công...,Tiêu đềMạnh mẽ vươn tới thành công. Mà nhẹ bỗn...,https://salt.tikicdn.com/cache/280x280/media/c...,17827,22900000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01
4,7608910,124742926,137665,"- Đóng gói chắc chắn, giao hàng nhanh - Sản ph...",10,2021-02-07 18:55:32,2021-02-07 18:55:32,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,4409272649267,<h3>Tiêu đề</h3><p>Mạnh mẽ vươn tới thành công...,Tiêu đềMạnh mẽ vươn tới thành công. Mà nhẹ bỗn...,https://salt.tikicdn.com/cache/280x280/media/c...,17827,22900000,1,1,2022-01-01 00:00:01,2022-01-01 00:00:01


In [6]:
product_ratings.shape

(44539, 18)

In [7]:
product_ratings=product_ratings.drop_duplicates()
product_ratings.shape

(44539, 18)

In [8]:
df = product_ratings[['userid','productid','rating', 'title', 'created_at_x']]

In [ ]:
df.head()

,userid,productid,rating,title,created_at_x
0,1110677,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2022-06-18 19:04:49
1,5249922,124742926,1,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2022-01-31 10:29:58
2,1464242,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-09-08 20:58:20
3,299031,124742926,6,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-08-06 23:23:46
4,137665,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-02-07 18:55:32


In [9]:
df.isnull().any().any()

False

In [10]:
df.isnull().sum().sum()

0

In [11]:
df=df.dropna()
df.shape

(44539, 5)

In [12]:
df.isnull().any().any()

False

In [13]:
len(df['userid'].unique())

39671

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44539 entries, 0 to 44538
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userid        44539 non-null  int32 
 1   productid     44539 non-null  int32 
 2   rating        44539 non-null  int8  
 3   title         44539 non-null  object
 4   created_at_x  44539 non-null  object
dtypes: int32(2), int8(1), object(2)
memory usage: 1.4+ MB


In [15]:
counts1=df['userid'].value_counts()
counts=df['productid'].value_counts()

In [ ]:
counts1

6599406     18
6106142     15
7178520     14
1064154     14
20434633    12
            ..
21922428     1
16561307     1
18230489     1
24905265     1
12401252     1
Name: userid, Length: 39671, dtype: int64

In [17]:
df1=df[df['userid'].isin(counts1[counts1 >= 3].index)]
df1.shape

(2839, 5)

In [18]:
df1=df1[df1['productid'].isin(counts[counts >= 3].index)]
df1.shape

(2830, 5)

In [19]:
len(df1['productid'].unique())

512

In [20]:
len(df1['userid'].unique())

785

In [21]:
df1.head()

,userid,productid,rating,title,created_at_x
2,1464242,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-09-08 20:58:20
4,137665,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-02-07 18:55:32
18,1137992,124742926,1,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-10-15 08:46:47
21,6284023,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2022-05-07 22:49:09
31,2091190,124742926,10,Apple MacBook Air 2020 13-inch (Apple M1 - 8GB...,2021-03-28 09:52:18


In [22]:
df1.shape

(2830, 5)

In [23]:
ratings_sum = pd.DataFrame(df1.groupby(['productid'])['rating'].count()).rename(columns = {'rating': 'rating_sum'})
top10 = ratings_sum.sort_values('rating_sum', ascending = False).head(10)
top10

,rating_sum
productid,
89683845,50
95469280,41
90369263,41
84285768,36
142545270,29
109393113,28
116382556,27
57694501,26
145495427,26


In [24]:
ratings_sum = pd.DataFrame(df1.groupby(['userid'])['rating'].count()).rename(columns = {'rating': 'rating_sum'})
top10 = ratings_sum.sort_values('rating_sum', ascending = False).head(50)
top10

,rating_sum
userid,
6599406,18
6106142,15
1064154,14
7178520,14
20434633,12
25565073,10
1021278,9
20435065,9
26126426,8


# Gợi ý sản phẩm
Xây dựng utility matrix với reviewerID và asin

In [25]:
df1.drop_duplicates(subset=['userid', 'productid'], keep='last', inplace=True)
df1[df1.duplicated(['userid', 'productid'], keep='last')]

,userid,productid,rating,title,created_at_x


In [26]:
df1.shape

(2562, 5)

In [27]:
# ratingsd=df1.pivot(index='productid',columns= 'userid',values='rating').fillna(0)

ratingsd = df1.pivot(index='userid', columns='productid', values='rating')
# ratingsd = ratingsd.fillna(ratingsd.mean())
ratingsd = ratingsd.fillna(0)
# ratingsd = ratingsd.T

# ratingsd.to_csv("tinhtay.csv")
ratingsd.head()

productid,54665,56033,56597,125781,126000,126697,150967,171745,180912,223377,...,171292910,171729035,172998120,173450461,174294337,175194008,175967811,176472430,176899932,176903003
userid,,,,,,,,,,,,,,,,,,,,,
3100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0


In [28]:
ratingsd.shape
ratingsd = ratingsd[ratingsd > 0] - ratingsd.mean()
ratingsd = ratingsd.fillna(0)
ratingsd.head()

productid,54665,56033,56597,125781,126000,126697,150967,171745,180912,223377,...,171292910,171729035,172998120,173450461,174294337,175194008,175967811,176472430,176899932,176903003
userid,,,,,,,,,,,,,,,,,,,,,
3100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
7864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.89172,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.872611,0.0


In [29]:
from sklearn.model_selection import train_test_split
traind, testd = train_test_split(ratingsd, test_size=0.30,random_state=42)

In [46]:
train = traind.values
test = testd.values

Tính similarity matrix theo công thức Cosine thông thường
$$ sim(u, u') = cos(\theta{}) = \frac{\textbf{r}_{u} \dot{} \textbf{r}_{u'}}{\| \textbf{r}_{u} \| \| \textbf{r}_{u'} \|} = \sum_{i} \frac{r_{ui}r_{u'i}}{\sqrt{\sum\limits_{i} r_{ui}^2} \sqrt{\sum\limits_{i} r_{u'i}^2} } $$

In [31]:
np.seterr(divide='ignore', invalid='ignore')
from IPython.display import clear_output

def cosine(a, b):
    p = a * b
    c, d = p/b, p/a
    c[np.isnan(c)] = 0
    d[np.isnan(d)] = 0
    norm = np.linalg.norm(c)*np.linalg.norm(d)
    if not norm:
        return 0
    return np.dot(a, b)/norm

def cosineMatrix(ratingMatrix):
    noitems = np.shape(ratingMatrix)[1]
    sim = np.zeros((noitems, noitems))
    for i in range(noitems):
        for j in range(i, noitems):
            sim[i][j] = cosine(ratingMatrix[:, i], ratingMatrix[:, j])
            sim[j][i] = sim[i][j]
        #print i
    return sim

def fast_similarity(ratings, epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

def adjusted_cosine(ratingMatrix):
    M = ratingMatrix
    M_u = M.mean(axis=1)
    item_mean_subtracted = M - M_u[:, None]
    similarity_matrix = 1 - squareform(pdist(item_mean_subtracted.T, 'cosine'))
    return similarity_matrix

def computeAdjCosSim(M):
    M = pd.DataFrame(M)
    sim_matrix = np.zeros((M.shape[1], M.shape[1]))
    M_u = M.mean(axis=1) #means

    count = 0
    for i in range(M.shape[1]):
        for j in range(M.shape[1]):
            count += 1
            clear_output(wait=True)
            print(M.shape[1])
            print(str(count / (M.shape[1]*M.shape[1]) * 100) + '%')
            if i == j:

                sim_matrix[i][j] = 1
            else:
                if i<j:

                    sum_num = sum_den1 = sum_den2 = 0
                    for k,row in M.loc[:,[i,j]].iterrows():

                        if ((M.loc[k,i] != 0) & (M.loc[k,j] != 0)):
                            num = (M[i][k]-M_u[k])*(M[j][k]-M_u[k])
                            den1= (M[i][k]-M_u[k])**2
                            den2= (M[j][k]-M_u[k])**2

                            sum_num = sum_num + num
                            sum_den1 = sum_den1 + den1
                            sum_den2 = sum_den2 + den2

                        else:
                            continue

                    den=(sum_den1**0.5)*(sum_den2**0.5)
                    if den!=0:
                        sim_matrix[i][j] = sum_num/den
                    else:
                        sim_matrix[i][j] = 0


                else:
                    sim_matrix[i][j] = sim_matrix[j][i]

#     return pd.DataFrame(sim_matrix)
    return sim_matrix

In [48]:
%%time
item_sim = adjusted_cosine(train)
# item_sim[:4, :4]
item_sim.shape

CPU times: user 61.5 ms, sys: 1.93 ms, total: 63.4 ms
Wall time: 62.2 ms


(512, 512)

In [49]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [50]:
%%time

item_prediction = predict_item(train, item_sim)

CPU times: user 23.3 ms, sys: 3.99 ms, total: 27.3 ms
Wall time: 21.7 ms


In [51]:
item_prediction[:4, :4]

array([[-0.03979171, -0.03455563, -0.02843848, -0.01735976],
       [-0.0226815 , -0.00570627,  0.00596683,  0.0193882 ],
       [-0.04133491, -0.03652191, -0.03053389, -0.01934356],
       [-0.01692367, -0.00888754, -0.00288677,  0.47175072]])

In [52]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as MAE
import math

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return math.sqrt(mean_squared_error(pred, actual))

def get_mae(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return MAE(actual, pred)

In [53]:
print('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

Item-based CF MSE: 9.20166331075397


In [54]:
item_sim = adjusted_cosine(ratingsd.values)
item_prediction = predict_item(ratingsd.values, item_sim)

In [55]:
item_prediction_df = pd.DataFrame(item_prediction, index=ratingsd.index, columns=ratingsd.columns)

item_prediction_df = item_prediction_df.T

item_prediction_df.head()

userid,3100,3570,3867,7864,9528,9777,9995,10241,11663,14303,...,27016952,27017500,27025799,27040250,27047760,27126910,27127007,27142490,27255003,27460300
productid,,,,,,,,,,,,,,,,,,,,,
54665,-0.025103,-0.030574,0.123356,-0.010968,0.112807,0.055068,-0.026683,0.105124,-0.011886,-0.031875,...,-0.089550,-0.111450,-0.089550,-0.066074,-0.039790,-0.111450,-0.111450,-0.047622,-0.089550,-0.045527
56033,-0.002497,-0.016744,0.084451,0.119034,0.117262,-0.017231,-0.021519,-0.025206,-0.011151,-0.035809,...,-0.182410,-0.226725,-0.182410,-0.133880,-0.067390,-0.226725,-0.226725,-0.019107,-0.182410,-0.053013
56597,0.011683,-0.004417,-0.051192,0.004013,-0.056517,-0.006907,-0.013252,-0.018543,-0.008033,-0.029580,...,-0.194428,-0.241576,-0.194428,-0.142491,-0.067801,-0.241576,-0.241576,0.002609,-0.194428,-0.044790
125781,-0.019935,-0.029389,0.488035,-0.009009,0.167766,0.239551,-0.028224,-0.029254,-0.013156,-0.037484,...,-0.135814,-0.168919,-0.135814,-0.099945,-0.055253,-0.168919,-0.168919,-0.043176,-0.135814,-0.054234
126000,-0.019703,-0.029148,-0.049999,-0.008910,0.155615,-0.027058,0.186281,0.266543,-0.013076,-0.037286,...,-0.135500,-0.168527,-0.135500,-0.099711,-0.055073,-0.168527,-0.168527,-0.042780,-0.135500,-0.053957


In [57]:
count = df1.groupby("productid", as_index=False).mean(numeric_only=True)
items_df = count[['productid']]

def recommend(predictions_df, itm_df, original_ratings_df, num_recommendations=10,ruserId='100031'):

    # Get and sort the user's predictions
    # User base

    # Item base
    sorted_user_predictions = predictions_df[ruserId].squeeze().sort_values(ascending=False)

    # Get the user's data and merge in the item information.
    user_data = original_ratings_df[original_ratings_df.userid == ruserId]
    #     user_full = (user_data.merge(itm_df, how = 'left', left_on = 'userid', right_on = 'productid').
    #                      sort_values(['rating'], ascending=False)
    #                  )

    prod_df = df1[['userid', 'productid', 'title', 'rating', 'created_at_x']]
    prod_df = prod_df[prod_df['userid'] == ruserId]
    prod_list = prod_df['title'].to_list()
    rating_list = prod_df['rating'].to_list()
    time_list = prod_df['created_at_x'].to_list()

    print ('User {0} đã đánh giá {1} sản phẩm:'.format(ruserId, user_data.shape[0]))
    for i, p in enumerate(prod_list):
        print('-------------------------------------------------')
        print(str(time_list[i]) + ' - ' +str(p))

    print('*************************************************')
    print ('Gợi ý {0} sản phẩm:'.format(num_recommendations))

    # Recommend the highest predicted rating items that the user hasn't bought yet.
    recommendations = (itm_df[~itm_df['productid'].isin(user_data['productid'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'productid',
               right_on = 'productid').
         rename(columns = {ruserId: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    ratings_sum = pd.DataFrame(df1.groupby(['productid'])['rating'].mean()).rename(columns = {'rating': 'rating_mean'})
    mean = ratings_sum.sort_values('rating_mean', ascending = False)

    topk=recommendations.merge(original_ratings_df, how='left', on='productid').drop_duplicates(
    ['productid', 'title'])[['productid', 'title']]

    topk1 = topk.merge(mean, how='left', on='productid')

    return topk1

In [61]:
recommend(item_prediction_df, items_df, df1, 5,137665)

User 137665 đã đánh giá 5 sản phẩm:
-------------------------------------------------
2021-02-07 18:55:32 - Apple MacBook Air 2020 13-inch (Apple M1 - 8GB/ 16GB - 256GB/ 512GB)
-------------------------------------------------
2020-12-17 06:45:51 - Tai Nghe Nhét Tai Mi Basic Xiaomi HSEJ03JY - Hàng Chính Hãng
-------------------------------------------------
2022-01-01 00:00:01 - Chuột không dây Logitech B175 - đầu thu USB 2.4Ghz, pin 1 năm, nhỏ gọn, thiết kế thuận cả 2 tay, phù hợp PC/ Laptop - Hàng chính hãng
-------------------------------------------------
2021-02-01 22:46:39 - Bàn Phím Bluetooth Logitech K380 - Hàng Chính Hãng
-------------------------------------------------
2021-02-01 22:41:36 - Chuột Không Dây Logitech M331 Silent - Hàng Chính Hãng
*************************************************
Gợi ý 5 sản phẩm:


,productid,title,rating_mean
0,62455045,Bàn phím game cơ học không dây Lightspeed Logi...,5.500000
1,26608647,Loa bluetooth Bose Home Speaker 300 - Hàng Chí...,10.000000
2,36331816,Loa Karaoke Bluetooth Xách Tay KM-S1 (Tặng Kèm...,10.000000
3,125781,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,7.000000
4,17954263,Màn hình máy tính AOC 24V2Q 24 inches IPS FHD ...,7.272727
